# ・QSS on IBMQ (Qiskit)

### Import modules. You need to install qiskit. Please refer to https://qiskit.org/documentation/install.html

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [2]:
import qiskit as qk
from qiskit import IBMQ, BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_overview

### Required IBMQ account from here. Please copy and paste your token into "mytoken".

In [3]:
IBMQ.update_account()

mytoken = ## your token ##
IBMQ.save_account(mytoken)

provider = IBMQ.load_account()
backends = provider.backends()
backends

Credentials already present. Set overwrite=True to overwrite.


Found 1 credentials.
No credentials available for updating could be found. No action will be performed.
Warnings:
The stored account with url "https://auth.quantum-computing.ibm.com/api" is already an IBM Q Experience v2 account.


/Users/NaoharuShimada/Library/Python/3.6/lib/python/site-packages/qiskit/providers/models/backendconfiguration.py:337: UserWarning: `dt` and `dtm` now have units of seconds(s) rather than nanoseconds(ns).
  warnings.warn('`dt` and `dtm` now have units of seconds(s) rather '


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [7]:
# Define circuits function
def Create_circuits(ave, P_qubits):

    # Define the Quantum and Classical Registers
    q = QuantumRegister(P_qubits+1, "q")
    c = ClassicalRegister(P_qubits+1, "c")

    # ave = 5/16.0
    theta = math.asin(math.sqrt(ave))*2
    # print(theta) # |0> -> √f|1>: θ rotation

    # ------- Build the circuits -------
    circuits = []
    
    middle = QuantumCircuit(q, c)
    for i in range(P_qubits):
        middle.h(q[i])
    middle.barrier()

    # Grover algorithm
    for k in range(P_qubits):
        for i in range(2**(k)):
            if (k==0):
                middle.cz(q[k],q[2])
            else:
                middle.z(q[2])
            middle.u3(-theta,0,0,q[2])
            middle.cz(q[k],q[2])
            middle.u3(theta,0,0,q[2])
        middle.barrier()

    # Measure a first qubit
    meas = QuantumCircuit(q, c)
    meas.measure(q[2],c[2])

    #QFT
    qft3 = QuantumCircuit(q, c, name="qft3")
    def qft(circ, q, n):
        """n-qubit QFT on q in circ."""
        for j in range(n):
            for k in range(j):
                circ.cu1(math.pi/float(2**(j-k)), q[k], q[j]) 
            circ.h(q[1-j])

    qft(qft3, q, P_qubits)

    # Measurement
    meas_QFT = QuantumCircuit(q, c)
    for i in range(P_qubits):
        meas_QFT.measure(q[i],c[i])
    
    # Compose all circuits above
    circuits.append(middle + meas + qft3 + meas_QFT)
    
    return circuits[0]

In [10]:
# Execute circuits function
def Run_circuits(ave, P_qubits, num_experiments, backend):
    print("Backend: " + backend.name())
    
    # Make circuits
    circuits = []
    for i in range(num_experiments):
        circuits.append(Create_circuits(ave, P_qubits))
    print("")
    print("The number of queries: ", (2**P_qubits-1)*2)
    print("The number of trials: ", num_experiments)
    print("")
    print("-------------------------------------------------------------------------------------------")
    print("Quantum circuit is here:")
    print(circuits[0].draw())
    print("-------------------------------------------------------------------------------------------")
    print("")

    job = execute(circuits, backend = backend, shots=1)
    job_monitor(job)
    job.error_message()
    result = job.result()

    error = 0.0
    for i in range(num_experiments):
        counts = result.get_counts(circuits[i])
        for a in counts.keys():
            b = [a[2],a[1]]
        #print(b)
        num = 0
        for j in range(len(b)):
            num += int(b[len(b)-1-j])*(2**j)
        #print(num)
        if (num<=2**(P_qubits-1)):
            num = 2**(P_qubits-1) - num
        else:
            temp = 2**(P_qubits) + 2**(P_qubits-1)
            num = temp - num
        #print(num)
        estimated_f = 1.0 - math.sin(num*math.pi/(2**P_qubits))**2
        print("Target:", ave, ",  Estimate: ", estimated_f)
        error += abs(estimated_f-ave)

    print("-------------------------------------------------------------------------------------------")
    print("Ave error:", error/num_experiments)

### Input variables. You can specify target mean value "ave" (0.0~1.0), and the number of register qubits "P_qubits" (up to 4 qubits) (refer to Figure 9 in our paper). The circuit is run by "num_experiments" times.

In [17]:
# input variables
ave = 0.5             # Target value
P_qubits = 2          # qubits for register ( the number of queries is defined by (2^P-1)*2 )
num_experiments = 10  # how many trials on the same circuits

### Execute calculation on simulator. You may confirm the circuit drawn below is compatible to one in Figure 9 (our paper). Resulting Error will be zero if ave=0.0, 0.5, or 1.0 as shown in Figure 7.

In [18]:
# Run on a simulator
Run_circuits(ave, P_qubits, num_experiments, backends[0])

Backend: ibmq_qasm_simulator

The number of queries:  6
The number of trials:  10

-------------------------------------------------------------------------------------------
Quantum circuit is here:
        ┌───┐ ░                                         ░      »
q_0: |0>┤ H ├─░──■───────────────────■──────────────────░──────»
        ├───┤ ░  │                   │                  ░      »
q_1: |0>┤ H ├─░──┼───────────────────┼──────────────────░──────»
        └───┘ ░  │ ┌───────────────┐ │ ┌──────────────┐ ░ ┌───┐»
q_2: |0>──────░──■─┤ U3(-pi/2,0,0) ├─■─┤ U3(pi/2,0,0) ├─░─┤ Z ├»
              ░    └───────────────┘   └──────────────┘ ░ └───┘»
 c_0: 0 ═══════════════════════════════════════════════════════»
                                                               »
 c_1: 0 ═══════════════════════════════════════════════════════»
                                                               »
 c_2: 0 ═══════════════════════════════════════════════════════»
                    

### Execute on a quantum computer. Resulting error will be much worser than on a simulator. 

In [19]:
# Run on an actual quantum computer
Run_circuits(ave, P_qubits, num_experiments, backends[1])

Backend: ibmqx2

The number of queries:  6
The number of trials:  10

-------------------------------------------------------------------------------------------
Quantum circuit is here:
        ┌───┐ ░                                         ░      »
q_0: |0>┤ H ├─░──■───────────────────■──────────────────░──────»
        ├───┤ ░  │                   │                  ░      »
q_1: |0>┤ H ├─░──┼───────────────────┼──────────────────░──────»
        └───┘ ░  │ ┌───────────────┐ │ ┌──────────────┐ ░ ┌───┐»
q_2: |0>──────░──■─┤ U3(-pi/2,0,0) ├─■─┤ U3(pi/2,0,0) ├─░─┤ Z ├»
              ░    └───────────────┘   └──────────────┘ ░ └───┘»
 c_0: 0 ═══════════════════════════════════════════════════════»
                                                               »
 c_1: 0 ═══════════════════════════════════════════════════════»
                                                               »
 c_2: 0 ═══════════════════════════════════════════════════════»
                                 